In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
text = """
artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.
"""


In [3]:
def preprocess_text(text):
    text = text.lower()
    chars = sorted(list(set(text)))
    
    char_to_idx = {ch:i for i,ch in enumerate(chars)}
    idx_to_char = {i:ch for i,ch in enumerate(chars)}
    
    encoded = [char_to_idx[ch] for ch in text]
    
    return encoded, char_to_idx, idx_to_char, len(chars)


In [4]:
def create_sequences(encoded_text, seq_length):
    inputs = []
    targets = []
    
    for i in range(len(encoded_text) - seq_length):
        inputs.append(encoded_text[i:i+seq_length])
        targets.append(encoded_text[i+seq_length])
        
    return inputs, targets


In [5]:
class ScratchGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(ScratchGRU, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        
        # Gates
        self.Wz = nn.Linear(hidden_size + hidden_size, hidden_size)
        self.Wr = nn.Linear(hidden_size + hidden_size, hidden_size)
        self.Wh = nn.Linear(hidden_size + hidden_size, hidden_size)
        
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x):
        batch_size, seq_len = x.size()
        
        h = torch.zeros(batch_size, self.hidden_size)
        
        for t in range(seq_len):
            x_embed = self.embedding(x[:, t])
            
            combined = torch.cat((h, x_embed), dim=1)
            
            z = torch.sigmoid(self.Wz(combined))
            r = torch.sigmoid(self.Wr(combined))
            
            combined_reset = torch.cat((r * h, x_embed), dim=1)
            h_tilde = torch.tanh(self.Wh(combined_reset))
            
            h = (1 - z) * h + z * h_tilde
        
        output = self.fc(h)
        return output


In [6]:
encoded, char_to_idx, idx_to_char, vocab_size = preprocess_text(text)

seq_length = 40
inputs, targets = create_sequences(encoded, seq_length)


In [7]:
hidden_size = 128
model = ScratchGRU(vocab_size, hidden_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)


In [8]:
epochs = 20

for epoch in range(epochs):
    total_loss = 0
    
    for i in range(len(inputs)):
        x = torch.tensor([inputs[i]])
        y = torch.tensor([targets[i]])
        
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(inputs):.4f}")


Epoch 1, Loss: 2.7124
Epoch 2, Loss: 2.3335
Epoch 3, Loss: 2.1649
Epoch 4, Loss: 2.0396
Epoch 5, Loss: 1.8988
Epoch 6, Loss: 1.8519
Epoch 7, Loss: 1.7310
Epoch 8, Loss: 1.6720
Epoch 9, Loss: 1.6577
Epoch 10, Loss: 1.5936
Epoch 11, Loss: 1.5231


KeyboardInterrupt: 

In [9]:
def generate_text(model, seed_text, seq_length, char_to_idx, idx_to_char, length=200):
    
    model.eval()
    generated = seed_text
    
    for _ in range(length):
        input_seq = generated[-seq_length:]
        input_indices = [char_to_idx[ch] for ch in input_seq]
        input_tensor = torch.tensor([input_indices])
        
        with torch.no_grad():
            output = model(input_tensor)
            prob = torch.softmax(output, dim=1)
            predicted_idx = torch.argmax(prob).item()
        
        generated += idx_to_char[predicted_idx]
    
    return generated


In [11]:
seed = "artificial intelligence "
print(generate_text(model, seed, seq_length, char_to_idx, idx_to_char, 30))


artificial intelligence enting.
aution sk usivn enhand


In [14]:
print(sum(p.numel() for p in model.parameters()))

106141


In [13]:
from torchinfo import summary

dummy_input = torch.randint(0, vocab_size, (1, seq_length), dtype=torch.long)

summary(model, input_data=dummy_input)



Layer (type:depth-idx)                   Output Shape              Param #
ScratchGRU                               [1, 29]                   --
├─Embedding: 1-1                         [1, 128]                  3,712
├─Linear: 1-2                            [1, 128]                  32,896
├─Linear: 1-3                            [1, 128]                  32,896
├─Linear: 1-4                            [1, 128]                  32,896
├─Embedding: 1-5                         [1, 128]                  (recursive)
├─Linear: 1-6                            [1, 128]                  (recursive)
├─Linear: 1-7                            [1, 128]                  (recursive)
├─Linear: 1-8                            [1, 128]                  (recursive)
├─Embedding: 1-9                         [1, 128]                  (recursive)
├─Linear: 1-10                           [1, 128]                  (recursive)
├─Linear: 1-11                           [1, 128]                  (recursive)
├─Linea